<a href="https://colab.research.google.com/github/ambgeo/geoquantificacao/blob/main/02_Cole%C3%A7%C3%B5es_de_Imagens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛰️ GEE + geemap: L9 SR, NDVI/MNDWI e Máscaras (Água/Vegetação)

Fluxo desta aula/notebook:
* 1) Autenticação
* 2) Definição da ROI usando `Map.user_roi`
* 3) Selecionar a **imagem Landsat 9 SR** com **menor percentual de nuvem** na ROI e **plota** as composições RGB (4‑3‑2) e SWIR/NIR/RED (6‑5‑4)
* 4) Calcular **NDVI** e **MNDWI**
* 5) Criar máscaras (vegetação `NDVI>0.6` e água `MNDWI>0.1`)
* 6) Exibir no mapa a imagem RGB (cor verdadeira) com as máscaras sobrepostas (água em azul, vegetação em verde)

## 1) Autenticação
Execute a célula abaixo para autenticar e iniciar a sessão do Earth Engine.
> Ajuste o `project` se necessário.

In [ ]:
import ee
try:
    ee.Authenticate()
except Exception as e:
    print('Se já autenticou anteriormente nesta máquina, pode ignorar este aviso:', e)
ee.Initialize(project='ee-scriptsambgeo')
print('GEE inicializado!')

## 2) Desenhar a ROI no `geemap` e capturar `Map.user_roi`
1. Rode a célula abaixo para abrir o mapa.
2. Use a ferramenta **Draw** (ícone do lápis) para desenhar um **polígono**.
3. Depois, rode a célula de **captura de ROI** para usar a geometria nas análises.

In [ ]:
# Se necessário: !pip -q install geemap folium ipyleaflet
import geemap
Map = geemap.Map(center=[-15, -55], zoom=4)
Map.addLayerControl()
Map

In [ ]:
# Capturar a ROI desenhada pelo usuário
roi = Map.user_roi
Map.centerObject(roi, 10)
Map.addLayer(roi, {"color":"red"}, 'ROI')
print('ROI definida!')
Map

## 3) Landsat 9 SR — menor nuvem e composições RGB e 6‑5‑4
- Usaremos **LANDSAT/LC09/C02/T1_L2** (Surface Reflectance / Level‑2).
- Escalonamento das bandas ópticas: `reflectance = DN * 0.0000275 + (-0.2)`.
- Máscara simples baseada em `QA_PIXEL` (nuvem, cirrus, cloud shadow, dilated).

In [ ]:
import datetime as dt

def scale_mask_l9(image):
    # Escala bands ópticas
    optical = image.select('SR_B.*').multiply(0.0000275).add(-0.2)
    img = image.addBands(optical, None, True)
    # QA mask (bits: 1 dilated, 2 cirrus, 3 cloud, 4 cloud shadow)
    qa = img.select('QA_PIXEL')
    dilated = qa.bitwiseAnd(1<<1).neq(0)
    cirrus  = qa.bitwiseAnd(1<<2).neq(0)
    cloud   = qa.bitwiseAnd(1<<3).neq(0)
    shadow  = qa.bitwiseAnd(1<<4).neq(0)
    mask = dilated.Or(cirrus).Or(cloud).Or(shadow).Not()
    return img.updateMask(mask)


In [ ]:
ee.Date(dt.datetime.now(dt.timezone.utc))

In [ ]:
# Janela temporal (últimos 18 meses, ajuste se quiser)
end = ee.Date(dt.datetime.now(dt.timezone.utc))
start = end.advance(-24, 'month')

col = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
       .filterBounds(roi)
       .filterDate(start, end)
       .sort('CLOUD_COVER'))

best = ee.Image(col.first())
best = scale_mask_l9(best)

print('Imagem selecionada:', best.get('LANDSAT_PRODUCT_ID').getInfo())
print('CLOUD_COVER:', best.get('CLOUD_COVER').getInfo())
print('Data:', ee.Date(best.get('system:time_start')).format('YYYY-MM-dd').getInfo())

In [ ]:
# Visualizações
viz_rgb = {"bands": ['SR_B4','SR_B3','SR_B2'], "min": 0.0, "max": 0.3}
viz_654 = {"bands": ['SR_B6','SR_B5','SR_B4'], "min": 0.0, "max": 0.3}

Map.addLayer(best, viz_rgb, 'L9 RGB (4-3-2)')
Map.addLayer(best, viz_654, 'L9 SWIR/NIR/RED (6-5-4)')
Map

## 4) Índices: NDVI e MNDWI
- **NDVI** = (NIR - Red) / (NIR + Red) → Bands: `SR_B5` e `SR_B4`
- **MNDWI** = (Green - SWIR1) / (Green + SWIR1) → Bands: `SR_B3` e `SR_B6`

In [ ]:
ndvi  = best.normalizedDifference(['SR_B5','SR_B4']).rename('NDVI')
mndwi = best.normalizedDifference(['SR_B3','SR_B6']).rename('MNDWI')
Map.addLayer(ndvi,  {"min":0, "max":1, "palette":['white','green']}, 'NDVI')
Map.addLayer(mndwi, {"min":-1, "max":1, "palette":['purple','cyan']}, 'MNDWI')
Map

## 5) Máscaras (vegetação e água) sobre RGB
- Vegetação: `NDVI > 0.6` (verde)
- Água: `MNDWI > 0.1` (azul)

In [ ]:
veg_mask   = ndvi.gt(0.7).selfMask()
water_mask = mndwi.gt(0.1).selfMask()

Map.addLayer(best, {"bands": ['SR_B4','SR_B3','SR_B2'], "min": 0.0, "max": 0.3}, 'RGB (base)')
Map.addLayer(water_mask, {"palette": ['#0000FF']}, 'Água (MNDWI>0.1)', True, 0.7)
Map.addLayer(veg_mask,   {"palette": ['#00FF00']}, 'Vegetação (NDVI>0.6)', True, 0.7)
Map

## 6) Área de corpos hídricos (km²) a partir do MNDWI
Calcula a soma da área (em m²) apenas onde `MNDWI > 0.1` e converte para km².

In [ ]:
# Área de água (km²) com base na máscara MNDWI>0.1
# (requer que 'water_mask' e 'roi' já estejam definidos)
limite = ee.Geometry(ee.Image(best).geometry().bounds(1))
water_area_img = ee.Image.pixelArea().updateMask(water_mask)  # m² por pixel onde há água

area_dict = water_area_img.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=limite,
    scale=30,          # resolução do Landsat 9
    maxPixels=1e13,
    tileScale=4
)

water_km2 = ee.Number(area_dict.get('area')).divide(1e6)  # m² -> km²
print('Área de corpos hídricos na ROI (km²):', water_km2.getInfo())


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 7) Exportar lâmina d’água como Shapefile (EPSG:4674)

# Opcional: se quiser usar só a ROI, troque para:
# export_geom = roi

# 7.1) Vetorizar a máscara de água (MNDWI > 0.1)
water_vectors = water_mask.rename('water').reduceToVectors(
    geometry=limite,
    scale=30,                       # Landsat 9
    geometryType='polygon',
    labelProperty='water',
    eightConnected=True,
    bestEffort=True,
    maxPixels=1e13,
    tileScale=4
)

# 7.2) Adicionar área (km²) a cada polígono
def add_area_km2(f):
    # área geodésica em m² -> km²
    return f.set({'area_km2': f.geometry().area(1).divide(1e6)})
water_vectors = water_vectors.map(add_area_km2)


# (Opcional) visualizar no mapa
# Map.addLayer(water_vectors_4674.style(color='0000FF', fillColor='0000FF66', width=1), {}, 'Água (vetor EPSG:4674)')

# 7.3) Exportar para o Google Drive (Shapefile)
task = ee.batch.Export.table.toDrive(
    collection=water_vectors,
    description='agua_mndwi',
    folder='Geoquanti',            # mude se quiser outra pasta no Drive
    fileNamePrefix='agua_mndwi',
    fileFormat='SHP',
    selectors=['water', 'area_km2']  # inclui o rótulo e a área; remova/adicione campos se quiser
)
task.start()
print('Exportação iniciada para Google Drive > pasta "Geoquanti".')
